In [2]:
import os
import copy
import json

import pandas as pd
import numpy as np

import pyproj # convert lat/lon into Plotly standard

import plotly.express as px
import plotly.graph_objects as go

pd.set_option('display.max_columns', 70)

import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = "vscode"


In [3]:
PWD = os.getcwd()
DATA_PATH = PWD + "/data/"

WALL_DATA_PATH = DATA_PATH + "wall-data.js"
CRIME_DATA_PATH = DATA_PATH + "crime-data.xlsx"
CITY_TO_POSTCODE_DATA_PATH = DATA_PATH + "city-state-postcode.csv"
POPULATION_TO_POSTCODE_DATA_PATH = DATA_PATH + "population-postcode.csv"
POLYGON_TO_POSTCODE_DATA_PATH = DATA_PATH + "postcode-polygon.geojson"


## Berlin walk data downloaded from All Trails

## Berlin districts downloaded from
https://daten.odis-berlin.de/en/dataset/bezirksgrenzen/
https://daten.odis-berlin.de/de/dataset/plz/
https://www.suche-postleitzahl.org/downloads

Tutorial: https://juanitorduz.github.io/germany_plots/

## Crime data downloaded as .xlsx from Berlin Police department
- https://www.berlin.de/polizei/

## !!! Load the Berlin Postcode Polygon data !!!

In [4]:
with open(POLYGON_TO_POSTCODE_DATA_PATH) as f:
    js_data = f.read()

# Extract the JSON part (assuming it's properly formatted)
# Here, we extract the part after 'export default '
json_data = js_data[js_data.find('{'):]  # Get everything from the first '{'

# Load the JSON data
berlin_district_polygon_geojson = json.loads(json_data)

# Extract coordinates from the GeoJSON
str(berlin_district_polygon_geojson)[0:800]

"{'type': 'FeatureCollection', 'name': 'lor_bezirksregionen_2021', 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::25833'}}, 'features': [{'type': 'Feature', 'properties': {'BZR_ID': '126011', 'BZR_NAME': 'MV Nord', 'BEZ': '12', 'STAND': '01.01.2021', 'GROESSE_m2': 2046186.803}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[388648.807, 5828699.019], [388617.259, 5828697.067], [388614.19, 5828696.877], [388614.724, 5828689.413], [388590.481, 5828687.729], [388590.091, 5828695.713], [388583.773, 5828695.283], [388572.168, 5828694.492], [388568.374, 5828694.233], [388498.747, 5828689.489], [388499.295, 5828682.554], [388456.032, 5828679.716], [388451.669, 5828679.357], [388411.406, 5828676.69], [388392.517, 5828675.443], [388367.545, 5828674.164], [388312.282, 5"

In [5]:
project = pyproj.Transformer.from_crs("epsg:32633", "epsg:4326", always_xy=True)


def convert_coords(geometry):

    # Create a deep copy to avoid modifying the original data
    new_coords = copy.deepcopy(geometry)
    for i, polygon in enumerate(geometry):
        for j, ring in enumerate(polygon):
            new_coords[i][j] = [list(project.transform(x, y)) for x, y in ring]
    return new_coords



In [6]:
df_polygon_postcode = pd.DataFrame(columns=['id', 'name', 'lat', 'lon', 'locations'])
df_rows = []

lat1, lon1 = [], []

for i, val in enumerate(berlin_district_polygon_geojson['features']):

    feature = berlin_district_polygon_geojson['features'][i]
    
    id = feature['properties']['BZR_ID']
    name = feature['properties']['BZR_NAME']
    polygon_type = feature['geometry']['type']

    coordinates = np.array(feature['geometry']['coordinates'])
    coordinates = np.squeeze(coordinates)
    temp_lon, temp_lat = map(list, zip(*coordinates))

    lat1 +=  temp_lat
    lon1 += temp_lon

    locations = convert_coords([[feature['geometry']['coordinates'][0][0][::15]]])

    new_row = {'id':int(id), 'name':name, 'lat':temp_lat, 'lon':temp_lon, 'locations':locations}
    df_rows.append(new_row)


df_polygon_postcode = pd.concat([df_polygon_postcode, pd.DataFrame(df_rows)], ignore_index=True)


df_polygon_postcode.head()


,id,name,lat,lon,locations
0,126011,MV Nord,"[5828699.019, 5828697.067, 5828696.877, 582868...","[388648.807, 388617.259, 388614.19, 388614.724...","[[[[13.355986367668118, 52.59689384421752], [1..."
1,76012,Marienfelde Nord,"[5810010.569, 5810014.673, 5810023.267, 581001...","[389279.457, 389285.014, 389282.138, 389312.91...","[[[[13.371515070055745, 52.42907847883873], [1..."
2,84012,Rudow,"[5806707.823, 5806703.743, 5806703.161, 580670...","[398905.383, 398894.61, 398892.74, 398884.092,...","[[[[13.514037840045324, 52.40126035318749], [1..."
3,121001,Ost 1 - Reginhardstraße,"[5825242.303, 5825225.904, 5825221.615, 582521...","[390101.227, 390098.267, 390097.493, 390095.54...","[[[[13.37856768870863, 52.566125839124986], [1..."
4,121002,Ost 2 - Alt-Reinickendorf,"[5825708.504, 5825698.948, 5825670.092, 582566...","[388905.024, 388907.816, 388916.245, 388916.88...","[[[[13.36077063969373, 52.57007237427104], [13..."


## !!! Load the Berlin Wall data !!!

In [7]:
with open(WALL_DATA_PATH) as f:
    js_data = f.read()

# Extract the JSON part (assuming it's properly formatted)
# Here, we extract the part after 'export default '
json_data = js_data[js_data.find('{'):]  # Get everything from the first '{'

# Load the JSON data
berlin_wall_geojson = json.loads(json_data)

# Extract coordinates from the GeoJSON
coordinates = berlin_wall_geojson['features'][0]['geometry']['coordinates'][0]

lon, lat, height = map(list, zip(*coordinates))

lon.append(lon[0])
lat.append(lat[0])
height.append(height[0])



In [8]:
# Earth radius in kilometers
R = 6371.0

def haversine_np(lat, lon):
    # Convert latitude and longitude from degrees to radians
    lat = np.radians(lat)
    lon = np.radians(lon)
    
    # Compute the differences between consecutive points
    dlat = lat[1:] - lat[:-1]
    dlon = lon[1:] - lon[:-1]
    
    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat[:-1]) * np.cos(lat[1:]) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Distance in kilometers
    distances = R * c
    
    # Insert zero at the start to represent distance from the first point
    distances = np.insert(distances, 0, 0.0)
    
    return distances

# Example latitude and longitude lists
latitude = np.array([...])  # Replace with your latitude values
longitude = np.array([...])  # Replace with your longitude values

# Calculate distances
dist = haversine_np(lat, lon)
tot_dist = np.cumsum(dist)

height = np.array(height)
tot_height = np.cumsum(height)


In [9]:
berlinwall_df = pd.DataFrame({'lat':lat, 'lon':lon, 'height':height, 'tot_height':tot_height, 'dist':dist, 'tot_dist':tot_dist})


# Create a scatter map with plotly
fig = px.scatter_mapbox(
    berlinwall_df,
    lat="lat",
    lon="lon",
    title="Berlin Wall",
    zoom=9,
    mapbox_style="carto-positron"
)

fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    title={"x": 0.5, "xanchor": "center"},
    height = 400,
    width = 400
)

fig.show()


## !!! Load the crime data !!!

The crime burden of the districts and district regions is shown on the basis of the frequency number (HZ). The HZ is a crime quotient that relates the number of cases that have become known to 100,000 inhabitants. This is the only way to compare areas with different population numbers.

The HZ is used as an indicator to express the threat caused by crime to an area. However, it should be noted that the HZ refers exclusively to the (reported) inhabitants of an area; other persons who frequent the area, e.g. customers, commuters or tourists, are not taken into account at the HZ. However, the crimes committed by this group of people are included in the calculation of the HZ. This means that the frequency figure for particularly attractive areas would actually be lower if the number of people exceeding the number of inhabitants could be quantified and included in the calculation. But that is not possible. High frequency figures are therefore not automatically a sign of negative living and quality of life, they can also be an expression of particular liveliness and popularity of areas.

In general, it is pointed out that the frequency figure alone should not be used without restriction for evaluation. This applies in particular to low-population district regions and to offences with basically low numbers of cases. Further explanations can be found at the top right under the button "Notes/Explanations".

In [43]:
# Load all sheets into a dictionary of DataFrames
crime_data = pd.read_excel(CRIME_DATA_PATH, skiprows=4, sheet_name=None)

# Display available sheet names
print(crime_data.keys())

crime_sheet_list = ['HZ_2013', 'HZ_2014', 'HZ_2015', 'HZ_2016', 'HZ_2017', 'HZ_2018', 'HZ_2019', 'HZ_2020', 'HZ_2021', 'HZ_2022']
# Access a specific sheet (e.g., 'Sheet1') or loop through all sheets if necessary
# df = crime_data['HZ_2022']  # replace 'Sheet1' with the actual sheet name

# Preview the DataFrame
crime_df = pd.DataFrame() 

for sheet_name in crime_sheet_list:
    # Extract the year from the key (e.g., 'HZ_2013' becomes '2013')
    year = int(sheet_name.split('_')[1])
    
    # Get the DataFrame for the current year and add a 'year' column
    df = crime_data[sheet_name]  # copy to avoid modifying the original data
    df['year'] = year
    
    # Append the current DataFrame to the combined DataFrame
    crime_df = pd.concat([crime_df, df], ignore_index=True)




crime_df.head()


dict_keys(['Titel', 'Inhaltsverzeichnis', 'Fallzahlen_2013', 'Fallzahlen_2014', 'Fallzahlen_2015', 'Fallzahlen_2016', 'Fallzahlen_2017', 'Fallzahlen_2018', 'Fallzahlen_2019', 'Fallzahlen_2020', 'Fallzahlen_2021', 'Fallzahlen_2022', 'HZ_2013', 'HZ_2014', 'HZ_2015', 'HZ_2016', 'HZ_2017', 'HZ_2018', 'HZ_2019', 'HZ_2020', 'HZ_2021', 'HZ_2022'])


,LOR-Schlüssel (Bezirksregion),Bezeichnung (Bezirksregion),Straftaten \n-insgesamt-,Raub,"Straßenraub,\nHandtaschen-raub",Körper-verletzungen \n-insgesamt-,Gefährl. und schwere Körper-verletzung,"Freiheits-beraubung, Nötigung,\nBedrohung, Nachstellung",Diebstahl \n-insgesamt-,Diebstahl von Kraftwagen,Diebstahl \nan/aus Kfz,Fahrrad-\ndiebstahl,Wohnraum-\neinbruch,Branddelikte \n-insgesamt-,Brand-\nstiftung,Sach-beschädigung -insgesamt-,Sach-beschädigung durch Graffiti,Rauschgift-delikte,Kieztaten,year
0,10000,Mitte,25303,319,167,2187,572,650,11442,186,1383,1032,436,95,35,1672,405,834,4746,2013
1,11001,Tiergarten Süd,36863,557,366,3522,1106,801,17399,99,2501,984,473,99,30,1677,152,732,6511,2013
2,11002,Regierungsviertel,75823,472,204,5213,1384,1330,39054,193,3014,2349,354,43,11,3701,1620,622,8903,2013
3,11003,Alexanderplatz,36892,382,228,2773,724,631,19528,160,1378,1673,583,81,35,2416,888,1007,5739,2013
4,11004,Brunnenstraße Süd,16464,120,34,812,184,237,9153,372,861,1294,515,23,11,1429,632,259,3618,2013


In [44]:
df.columns

Index(['LOR-Schlüssel (Bezirksregion)', 'Bezeichnung (Bezirksregion)',
       'Straftaten \n-insgesamt-', 'Raub', 'Straßenraub,\nHandtaschen-raub',
       'Körper-verletzungen \n-insgesamt-',
       'Gefährl. und schwere Körper-verletzung',
       'Freiheits-beraubung, Nötigung,\nBedrohung, Nachstellung',
       'Diebstahl \n-insgesamt-', 'Diebstahl von Kraftwagen',
       'Diebstahl \nan/aus Kfz', 'Fahrrad-\ndiebstahl', 'Wohnraum-\neinbruch',
       'Branddelikte \n-insgesamt-', 'Brand-\nstiftung',
       'Sach-beschädigung -insgesamt-', 'Sach-beschädigung durch Graffiti',
       'Rauschgift-delikte', 'Kieztaten', 'year'],
      dtype='object')

In [45]:
german_col_name = list(crime_df.columns)

english_col_name = ['LOR key (district region)', 
'Name (district region)',
'Crimes Total', 
'Robbery', 
'Street Robbery',
'Physical Injuries Total',
'Dangerous Injuries',
'Threats and Stalking',
'Theft Total', 
'Theft of Cars',
'Theft From Cars', 
'Theft of Bicycle', 
'Burglary',
'Arson Total', 
'Arson Starting',
'Damage to Property Total', 
'Damage to Property by Graffiti',
'Drug Offences', 
'Neighborhood Crimes',
'Year']

rename_dict = dict(zip(german_col_name, english_col_name))

In [47]:
# Perform the merge operation
berlin_df = pd.merge(
    df_polygon_postcode,
    crime_df, 
    how='left',
    left_on='id',  # Matching on 'id' from df_polygon_postcode
    right_on='LOR-Schlüssel (Bezirksregion)'  # Matching on 'LOR-Schlüssel (Bezirksregion)' from df
)

# Convert multiple columns to numeric
columns_to_convert = ['id',
       'Straftaten \n-insgesamt-', 'Raub', 'Straßenraub,\nHandtaschen-raub',
       'Körper-verletzungen \n-insgesamt-',
       'Gefährl. und schwere Körper-verletzung',
       'Freiheits-beraubung, Nötigung,\nBedrohung, Nachstellung',
       'Diebstahl \n-insgesamt-', 'Diebstahl von Kraftwagen',
       'Diebstahl \nan/aus Kfz', 'Fahrrad-\ndiebstahl', 'Wohnraum-\neinbruch',
       'Branddelikte \n-insgesamt-', 'Brand-\nstiftung',
       'Sach-beschädigung -insgesamt-', 'Sach-beschädigung durch Graffiti',
       'Rauschgift-delikte', 'Kieztaten', 'year']

berlin_df[columns_to_convert] = berlin_df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

berlin_df.rename(columns=rename_dict, inplace=True)

berlin_df.head()

,id,name,lat,lon,locations,LOR key (district region),Name (district region),Crimes Total,Robbery,Street Robbery,Physical Injuries Total,Dangerous Injuries,Threats and Stalking,Theft Total,Theft of Cars,Theft From Cars,Theft of Bicycle,Burglary,Arson Total,Arson Starting,Damage to Property Total,Damage to Property by Graffiti,Drug Offences,Neighborhood Crimes,Year
0,126011,MV Nord,"[5828699.019, 5828697.067, 5828696.877, 582868...","[388648.807, 388617.259, 388614.19, 388614.724...","[[[[13.355986367668118, 52.59689384421752], [1...",126011,MV Nord,13914.0,291.0,187.0,1856.0,461.0,672.0,6343.0,141.0,955.0,345.0,174.0,104.0,33.0,1249.0,116.0,461.0,4570.0,2013
1,126011,MV Nord,"[5828699.019, 5828697.067, 5828696.877, 582868...","[388648.807, 388617.259, 388614.19, 388614.724...","[[[[13.355986367668118, 52.59689384421752], [1...",126011,MV Nord,13478.0,219.0,118.0,1719.0,435.0,642.0,6255.0,138.0,984.0,443.0,150.0,179.0,81.0,1146.0,102.0,297.0,4715.0,2014
2,126011,MV Nord,"[5828699.019, 5828697.067, 5828696.877, 582868...","[388648.807, 388617.259, 388614.19, 388614.724...","[[[[13.355986367668118, 52.59689384421752], [1...",126011,MV Nord,13613.0,170.0,97.0,1500.0,392.0,542.0,6709.0,231.0,1051.0,368.0,89.0,109.0,28.0,902.0,65.0,307.0,4485.0,2015
3,126011,MV Nord,"[5828699.019, 5828697.067, 5828696.877, 582868...","[388648.807, 388617.259, 388614.19, 388614.724...","[[[[13.355986367668118, 52.59689384421752], [1...",126011,MV Nord,12364.0,185.0,118.0,1458.0,436.0,515.0,5657.0,161.0,963.0,365.0,110.0,98.0,43.0,1116.0,244.0,334.0,3567.0,2016
4,126011,MV Nord,"[5828699.019, 5828697.067, 5828696.877, 582868...","[388648.807, 388617.259, 388614.19, 388614.724...","[[[[13.355986367668118, 52.59689384421752], [1...",126011,MV Nord,9768.0,123.0,69.0,1406.0,272.0,487.0,4181.0,157.0,832.0,264.0,61.0,115.0,61.0,820.0,149.0,218.0,2705.0,2017


In [48]:
# Create a GeoJSON-like structure
features = []
for index, row in berlin_df.drop_duplicates(subset='id', keep='last').iterrows():
    features.append({
        'type': 'Feature',
        'geometry': {
            'type': 'MultiPolygon',
            'coordinates': row['locations']  # Use the coordinates from the DataFrame
        },
        'properties': {
            'id': int(row['id']),
            'name': row['name'],
        }
    })

geojson_data = {
    'type': 'FeatureCollection',
    'features': features
}

# Plotting everything together

In [49]:
fig = px.choropleth_mapbox(
    berlin_df.drop_duplicates(subset='id', keep='last'),
    geojson=geojson_data,
    locations='id',  # Column in DataFrame that matches 'id' in GeoJSON
    featureidkey="properties.id", 
    color='Crimes Total',    # Column you want to color by
    color_continuous_scale="Reds",
    hover_data={
        'id': True,          # Include 'id' in hover info
        'name': True,        # Include 'name' in hover info
        'Crimes Total': True         # Include 'Raub' in hover info
    },
    # mapbox_style="carto-positron",
    mapbox_style="open-street-map", # more vibrant
    zoom=9,
    center={"lat": 52.5200, "lon": 13.4050},  # Center on Berlin coordinates
    opacity=0.6,
    title="Choropleth Map of Berlin Postcode Districts by Raub Incidents"
)


fig.add_trace(go.Scattermapbox(
    lat=berlinwall_df['lat'],
    lon=berlinwall_df['lon'],
    mode='lines',
    line=dict(color='black', width=3),  # Set line color and width
    name='Berlin Wall Boundary',
    
))

fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    title={"x": 0.5, "xanchor": "center"},
    height=800,
    width=1000,
)

fig.show()

# fig.write_html(f"{PWD}/figures/Berlin-Crime-vs-Wall.html")

In [50]:
slider_col_name = ['Crimes Total', 
'Robbery', 
'Physical Injuries Total',
'Dangerous Injuries',
'Threats and Stalking',
'Theft of Cars',
'Theft From Cars', 
'Theft of Bicycle', 
'Burglary',
'Arson Total', 
'Damage to Property Total', 
'Damage to Property by Graffiti',
'Drug Offences']


# Create a base figure
fig = go.Figure()


fig.add_trace(go.Scattermapbox(
    lat=berlinwall_df['lat'],
    lon=berlinwall_df['lon'],
    mode='lines',
    line=dict(color='black', width=3),  # Set line color and width
    name='Berlin Wall Boundary',
    visible=True
    
))

# Add traces for each column to the figure
for column in slider_col_name:
    fig.add_trace(go.Choroplethmapbox(
        geojson=geojson_data,
        locations=berlin_df.drop_duplicates(subset='id', keep='last')['id'],
        featureidkey="properties.id",
        z=berlin_df.drop_duplicates(subset='id', keep='last')[column],
        colorscale='RdYlGn',
        reversescale=True,
        marker_opacity=0.5,
        name=column,
        visible=False,  # Set all to invisible initially
        hoverinfo='text',  # Show text on hover
        hovertemplate=(
            'Area: %{customdata}<br>'  # Use customdata for the area name
            f'{column}: %{{z}}<extra></extra>'
        ),
        customdata=berlin_df.drop_duplicates(subset='id', keep='last')['name']
    ))

# Set the first trace to be visible
fig.data[1].visible = True

# Create sliders
sliders = [
    {
        'active': 0,
        'currentvalue': {
            'prefix': 'Current column: ',
            'visible': True,
            'xanchor': 'right'
        },
        'pad': {'b': 5},
        'len': 0.95,
        'x': 0.05,
        'y': -0.1,
        'steps': [
            {
                'method': 'update',
                'label': column,
                'args': [
                    {'visible': [True] + [i == idx for i in range(len(slider_col_name))]},
                    {'title': f'{column} per 100,000 people in Berlin'}
                ]
            }
            for idx, column in enumerate(slider_col_name)
        ]
    }
]

# Update layout with sliders
fig.update_layout(
    title=f'{slider_col_name[0]} per 100,000 people in Berlin',
    mapbox_style="open-street-map",
    mapbox_zoom=9,
    mapbox_center={"lat": 52.5200, "lon": 13.4050},
    sliders=sliders,
)




fig.write_html(f"{PWD}/figures/Berlin-Crime-vs-Wall-Slider.html")


### Adding year as a second slider

In [57]:
# Define available years and crime columns
years = berlin_df['Year'].unique()

slider_col_name = ['Crimes Total', 
'Robbery', 
'Physical Injuries Total',
'Dangerous Injuries',
'Threats and Stalking',
'Theft of Cars',
'Theft From Cars', 
'Theft of Bicycle', 
'Burglary',
'Arson Total', 
'Damage to Property Total', 
'Damage to Property by Graffiti',
'Drug Offences']

# Create a base figure
fig = go.Figure()

# Add Berlin Wall Boundary
fig.add_trace(go.Scattermapbox(
    lat=berlinwall_df['lat'],
    lon=berlinwall_df['lon'],
    mode='lines',
    line=dict(color='black', width=3),  # Set line color and width
    name='Berlin Wall Boundary',
    visible=True
))

trace_idx = 1

# Add traces for each year and crime type
for year_idx, year in enumerate(years):
    for crime_idx, column in enumerate(slider_col_name):
        is_initial_trace = (year_idx == 0 and crime_idx == 0)
        fig.add_trace(go.Choroplethmapbox(
            geojson=geojson_data,
            locations=berlin_df[berlin_df['Year'] == year].drop_duplicates(subset='id', keep='last')['id'],
            featureidkey="properties.id",
            z=berlin_df[berlin_df['Year'] == year].drop_duplicates(subset='id', keep='last')[column],
            colorscale='RdYlGn',
            reversescale=True,
            marker_opacity=0.5,
            name=f"{column} ({year})",
            visible=False,
            hoverinfo='text',
            hovertemplate=(
                'Area: %{customdata}<br>'
                f'{column}: %{{z}}<extra></extra>'
            ),
            customdata=berlin_df.drop_duplicates(subset='id', keep='last')['name']
        ))

        trace_idx += 1

# Set the initial trace to visible
# fig.data[1].visible = True


# Create dropdown menu for crime types
dropdown_buttons = []
for crime_idx, column in enumerate(slider_col_name):
    button = {
        'method': 'update',
        'label': column,
        'args': [
            {
                'visible': [
                    (i % len(years) == 0 and i // len(years) == crime_idx)  # Only show traces for selected crime type and first year
                    for i in range(len(slider_col_name) * len(years))
                ]
            },
            {'title': f'{column} per 100,000 people in Berlin'}
        ]
    }
    dropdown_buttons.append(button)

# Create year slider
year_slider_steps = []
for year_idx, year in enumerate(years):
    step = {
        'method': 'update',
        'label': str(year),
        'args': [
            {
                'visible': [
                    (i // len(slider_col_name) % len(years) == year_idx)  # Only show traces for selected year
                    for i in range(len(slider_col_name) * len(years))
                ]
            },
            {'title': f'Crime Data for {year}'}
        ]
    }
    year_slider_steps.append(step)

# Define layout with dropdown and slider
fig.update_layout(
    title=f'{slider_col_name[0]} per 100,000 people in Berlin',
    mapbox_style="open-street-map",
    mapbox_zoom=9,
    mapbox_center={"lat": 52.5200, "lon": 13.4050},
    updatemenus=[{
        'buttons': dropdown_buttons,
        'direction': 'down',
        'showactive': True,
        'x': 0.05,
        'xanchor': 'left',
        'y': 1.15,
        'yanchor': 'top'
    }],
    sliders=[{
        'active': 0,
        'currentvalue': {'prefix': 'Year: ', 'visible': True, 'xanchor': 'right'},
        'pad': {'b': 50},
        'len': 0.95,
        'x': 0.05,
        'y': -0.1,
        'steps': year_slider_steps
    }]
)



fig.write_html(f"{PWD}/figures/Berlin-Crime-vs-Year-Slider.html")


In [55]:
year_slider

{'active': 0,
 'currentvalue': {'prefix': 'Year: ', 'visible': True, 'xanchor': 'right'},
 'pad': {'b': 50},
 'len': 0.95,
 'x': 0.05,
 'y': -0.2,
 'steps': [{'method': 'update',
   'label': '2013',
   'args': [{'visible': [True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
      False,
   